In [1]:
import polars as pl
import numpy as np
import yfinance as yf

from hedging_strategy import hedging_strategies, portfolio, risk

In [2]:
spy = yf.download('SPY')[['Adj Close']] * 100 # assume we hold 100 shares
spy_pl = (
    pl.from_pandas(
        spy.reset_index()
    )
    .with_columns(
        pl.col('Date').cast(pl.Date)
    )
    .rename(
        {
            'Adj Close' : 'Holding'
        }
    )
)

[*********************100%***********************]  1 of 1 completed


In [3]:
option_data = (
    pl.read_parquet(
        'Data/SPX_options_1996_2024.parquet'
    )
    .with_columns(
        pl.col('date').cast(pl.Date),
        pl.col('exdate').cast(pl.Date)
    )
)

In [10]:
hedge_date = '2020-02-10'
hedge = hedging_strategies.hedging(
    hedge_date = hedge_date,
    holding_period = 180,
    options = option_data,
    min_dte = 300,
    close = 3380
)

long_put_value = hedge.buy_put()
short_call_value = hedge.short_call()


port = portfolio.portfolio(
    hedge_date = hedge_date,
    holding_value = spy_pl
)

do_nothing = (
    port
    .compute_portfolio()
)

long_put = (
    port
    .compute_portfolio(
        hedge_value = (
            long_put_value
            .select(
                'date', 'hedge_value'
            )
        )
    )
)

short_call = (
    port
    .compute_portfolio(
        hedge_value = (
            short_call_value
            .select(
                'date', 'hedge_value'
            )
        )
    )
)


returns_list = [
    pl.Series(name = 'do_nothing', values = do_nothing['returns'].to_numpy().flatten()[1:]),
    pl.Series(name = 'long_put', values = long_put['returns'].to_numpy().flatten()[1:]),
    pl.Series(name = 'short_call', values = short_call['returns'].to_numpy().flatten()[1:])
]

In [11]:
risk_info = (
    risk.risk(
        returns = returns_list
    )
)

print(risk_info.compute_var(95))

(
    risk_info
    .visualize_returns(
        bins = 1/90,
        title = 'Russia Invasion',
        n_rows = 1, 
        height = 400, 
        width = 1500
    )
)

shape: (3, 2)
┌────────────┬───────────┐
│ strategy   ┆ 95_var    │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ do_nothing ┆ -0.019675 │
│ long_put   ┆ -0.097374 │
│ short_call ┆ -0.024623 │
└────────────┴───────────┘
